In [1]:
%cd "/content/drive/MyDrive/Master ML/NLP Project"

[Errno 2] No such file or directory: '/content/drive/MyDrive/Master ML/NLP Project'
/Users/adamameen/Documents/Masters/2nd Year/FYP


In [4]:
!pwd

/content/drive/MyDrive/Master ML/NLP Project


In [5]:
!ls

'Ratings Predictor.ipynb'   tripadvisor_hotel_reviews.csv
 Text_Summarization.ipynb   tripadvisor_hotel_reviews.xlsx


In [44]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [70]:
#importing the required library
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.metrics import f1_score, accuracy_score
from sklearn import model_selection
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
stop = stopwords.words('english')

In [7]:
#loading data
data_df = pd.read_csv('tripadvisor_hotel_reviews.csv')

In [14]:
#checking whether there are any missing values
print(data_df.isnull().sum())

Review       0
Rating       0
Sentiment    0
dtype: int64


In [10]:
#checking the clas counts
data_df.Rating.value_counts()

5    9054
4    6039
3    2184
2    1793
1    1421
Name: Rating, dtype: int64

In [15]:
#creating a new dataframe with only the required columns
df = data_df[['Review', 'Rating']]
df.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [72]:
#now splitting the data into to Train and test test
X = df['Review']
y = df['Rating']
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42,test_size=0.3)

In [73]:
lemmatizer = WordNetLemmatizer()
def preprocessing(x):
  #first we make text to lowercase 
  x = x.apply(lambda x: " ".join(x.lower() for x in x.split()))
  #remove punctuation
  x = x.str.replace('[^\w\s]','')
  #removing stop workds
  x = x.apply(lambda x: " ".join(x for x in x.split() if x not in stop))
  #removing digits
  x = x.str.replace('\d+', '')
  #lemmatizing
  x = [lemmatizer.lemmatize(row) for row in x]
  return x

preprocessor = FunctionTransformer(preprocessing)

In [74]:
tfidf = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,2), max_features=5000)
nb = MultinomialNB(alpha = 0.6)
pipeline = Pipeline([
    ('Preprocessor',preprocessor),
    ('Tf-Idf', tfidf),
    ('classifier',nb )
])
pipeline.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  # Remove the CWD from sys.path while we load stuff.


Pipeline(steps=[('Preprocessor',
                 FunctionTransformer(func=<function preprocessing at 0x7f31b60c7f80>)),
                ('Tf-Idf',
                 TfidfVectorizer(max_features=5000, ngram_range=(1, 2),
                                 token_pattern='\\w{1,}')),
                ('classifier', MultinomialNB(alpha=0.6))])

In [75]:
pip_pred = pipeline.predict(X_test)
print(metrics.classification_report(y_test, pip_pred))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  # Remove the CWD from sys.path while we load stuff.


              precision    recall  f1-score   support

           1       0.76      0.50      0.60       432
           2       0.39      0.31      0.35       523
           3       0.34      0.04      0.07       645
           4       0.46      0.43      0.44      1887
           5       0.64      0.88      0.74      2661

    accuracy                           0.58      6148
   macro avg       0.52      0.43      0.44      6148
weighted avg       0.54      0.58      0.54      6148



In [79]:
pipeline.predict(pd.Series(['Hey this is the worst place i have been.','this is the best of the best. I love it','its okay, not the best but its okay']))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  # Remove the CWD from sys.path while we load stuff.


array([1, 5, 4])

In [80]:
import joblib
joblib.dump(pipeline, 'Rating_predictor.pkl', compress = 1)

['Rating_predictor.pkl']